# 00_Setup

本次處理資料程式碼，基本上全程處理的過程中，欄位都是以中文方式讀取。  
大家也請協助先核對是否有下載的程式封包 以及 對應儲存的資料夾結構，也可以調整對應的位置  
1. ProcessBasic ：https://github.com/KaiChiehKJ/THI-ProcessTool/blob/main/ProcessBasic.py  （為B_895 提供的程式封包）  

2. 資料夾結構如下：  （命名原則：是原始資料，需要在資料夾後綴加上"資料"；路段、路口需標明）  


交通調查/  
├── 01_交通調查計畫     <span style="background-color: gray; color: white; font-size: 12px; ">planfolder</span>  
├── 02_交通調查公司資料  <span style="background-color: gray; color: white; font-size: 12px;">investigatefolder</span>  
│   └── *某調查期間*  # 如：清明連假、平日、假日  
│       ├── 交通量資料  
│       │   ├── 路口  
│       │   └── 路段  
│       └── 速率資料  
├── 04_調查資料分析  
│   └── Code   # 目前程式所存放資料夾 (當前資料夾)  <span style="background-color: gray; color: white; font-size: 12px;">os.getcwd()</span>   
│       │   └── 統整調查資料(集賢).ipynb  # 負責營運資料與票證資料的比對與計算  
│   └── 01_調查初步分析結果  <span style="background-color: gray; color: white; font-size: 12px;">step1folder</span>    
│   └── 02_整併交通量資料  <span style="background-color: gray; color: white; font-size: 12px;">step2folder</span>    


In [1]:
from ProcessBasic import * # https://github.com/KaiChiehKJ/THI-ProcessTool/blob/main/ProcessBasic.py

In [2]:
def get_filename_withoutprojectname(path, step = 0):
    """
    從檔案路徑中提取不包含專案名稱的檔名。
    
    Args:
        path (str): 檔案的完整路徑。
    
    Returns:
        str: 移除專案資料夾後的檔案名稱。
    """
    project_folder = get_projectfolderpath(step)
    relative_path = path.replace(project_folder, "~")
    return relative_path

def get_peak_data(df, group_by, sum_by, hourcolumn):
    """
    取得指定資料欄位中的尖峰時段資料，並返回最大PCU值對應的資料。

    Args:
        df (DataFrame): 要處理的資料集。
        group_by (str): 用來分組的欄位名稱。
        sum_by (str): 用來求最大值的欄位名稱。
        hourcolumn (str): 代表小時的欄位名稱。

    Returns:
        DataFrame: 包含每組尖峰時段資料及其對應的最大PCU值的資料集。
    """

    # 取得每組的最大 PCU 值對應的索引
    idx = df.groupby(group_by)[sum_by].idxmax()
    # 取出尖峰時段資料
    peak_hour = df.loc[idx].copy()
    # 重新命名欄位
    peak_hour = peak_hour.rename(columns={sum_by: '尖峰小時PCU', hourcolumn: '尖峰時段'})
    return peak_hour.reset_index(drop=True)

def get_peak_AMPM(df, group_by, sum_by, hourcolumn):
    """
    取得指定資料欄位中的晨峰及昏峰資料，並返回最大PCU值對應的資料。

    Args:
        df (DataFrame): 要處理的資料集。
        group_by (str): 用來分組的欄位名稱。
        sum_by (str): 用來求最大值的欄位名稱。
        hourcolumn (str): 代表小時的欄位名稱。

    Returns:
        DataFrame: 包含每組尖峰時段資料及其對應的最大PCU值的資料集。
    """

    df_AM = df[df[hourcolumn] <= 12].reset_index(drop = True)
    df_PM = df[df[hourcolumn] > 12].reset_index(drop = True)
    df_AM_peak = get_peak_data(df = df_AM, group_by=group_by, sum_by=sum_by, hourcolumn=hourcolumn).rename(columns = {'尖峰小時PCU':'晨峰小時PCU'})
    df_PM_peak = get_peak_data(df = df_PM, group_by=group_by, sum_by=sum_by, hourcolumn=hourcolumn).rename(columns = {'尖峰小時PCU':'昏峰小時PCU'})
    return df_AM_peak, df_PM_peak

def get_peak_percent(df, group_by, sum_by, hourcolumn):

    """
    取得指定資料欄位中的尖峰時段資料，並計算尖峰小時比例返回最大PCU值對應的資料。
    須包含 "get_peak_data" 這個函數。

    Args:
        df (DataFrame): 要處理的資料集。
        group_by (str): 用來分組的欄位名稱。
        sum_by (str): 用來求最大值的欄位名稱。
        hourcolumn (str): 代表小時的欄位名稱。

    Returns:
        DataFrame: 包含每組尖峰時段資料及其對應的最大PCU值的資料集。
    """

    df_peakdata = get_peak_data(df, group_by=group_by, sum_by=sum_by, hourcolumn=hourcolumn)
    df_daily = df.groupby(group_by).agg({sum_by:'sum'}).reset_index()
    df_daily = pd.merge(df_peakdata, df_daily)
    df_daily['尖峰率'] = df_daily['尖峰小時PCU'] / df_daily['PCU']
    df_daily = df_daily.drop(columns = ['尖峰時段', sum_by])
    output = pd.merge(df, df_daily, on = group_by, how='left')
    # 刪除所有以 '_y' 結尾的欄位
    output = output.drop(columns=[col for col in output.columns if col.endswith('_y')])
    # 將所有以 '_x' 結尾的欄位名稱中的 '_x' 改為空字符，或者你也可以改為其他文字
    output.columns = [col.replace('_x', '') for col in output.columns]
    output = output.drop_duplicates()
    return output    


In [3]:
projectfolder = get_projectfolderpath() # 專案資料夾

planfolder = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '01_交通調查計畫'))
investigatefolder = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '02_交通調查公司資料'))
step1folder = create_folder(os.path.join(os.getcwd(), '..', '..', '04_調查資料分析', '01_調查初步分析結果'))
step2folder = create_folder(os.path.join(os.getcwd(), '..', '..', '04_調查資料分析', '02_整併交通量資料'))

PCE = {'小型車':1,
       '大型車':1.8,
       '機車':0.42, 
       '小客車':1, 
       '小貨車':1,
       '大客車':1.8,
       '大貨車':1.8} # 先用模型常用值進行計算 (2022 公路容量手冊沒有特別細部說明)

# 01_調查資料處理

## 01-01 速率調查

如果資料本身可以直接用，那就直接執行 *organize_speedinvestigation_table*  

In [4]:
def speed_organize_step1(excelfile, columns_name = ['區間', '路段長度','方向0_旅行速率', '方向0_服務水準', '方向1_旅行速率', '方向1_服務水準']):
    df = pd.read_excel(excelfile, sheet_name='資料整理')
    # excelname = get_filename(excelfile)
    excelname = get_filename_withoutprojectname(excelfile, step=0)
    values = list(df.iloc[2, [0, 7, 14, 21]])

    df = pd.read_excel(excelfile, sheet_name='資料整理', skiprows=5)
    df_workday_AM = df.iloc[:,:6]
    df_workday_AM.columns = columns_name
    df_workday_AM = df_workday_AM.reset_index(names = '對應速率調查路段編號')
    df_workday_AM['時間'] = values[0]

    df_workday_Other = df.iloc[:,7:13] # 欄位需要隨著每次提供檔案調整
    df_workday_Other.columns = columns_name
    df_workday_Other = df_workday_Other.reset_index(names = '對應速率調查路段編號')
    df_workday_Other['時間'] = values[1]

    df_workday_PM = df.iloc[:,14:20]
    df_workday_PM.columns = columns_name
    df_workday_PM = df_workday_PM.reset_index(names = '對應速率調查路段編號')
    df_workday_PM['時間'] = values[2]

    df_weekend_Peak = df.iloc[:,21:27]
    df_weekend_Peak.columns = columns_name
    df_weekend_Peak = df_weekend_Peak.reset_index(names = '對應速率調查路段編號')
    df_weekend_Peak['時間'] = values[3]

    combinded_df = pd.concat([df_workday_AM, df_workday_Other, df_workday_PM, df_weekend_Peak], ignore_index=True)
    combinded_df = combinded_df[(combinded_df['方向0_旅行速率']!="-") & (combinded_df['區間'] != '全線')]
    combinded_df['資料來源'] = excelname

    return combinded_df

def speed_organize_step2(df):
    columnsname = ['對應速率調查路段編號', '區間', '路段長度', '旅行速率', '服務水準', '時間', '資料來源']

    df0 = df.drop(columns = ['方向1_旅行速率','方向1_服務水準'])
    df1 = df.drop(columns = ['方向0_旅行速率', '方向0_服務水準'])
    
    df0.columns = columnsname
    df1.columns = columnsname

    df0['對應速率調查方向'] = 0
    df1['對應速率調查方向'] = 1

    output_df = pd.concat([df0, df1], ignore_index=True)

    output_df["對應速率路線編號"] = output_df["資料來源"].apply(lambda x: os.path.basename(x))
    output_df["對應速率路線編號"] = output_df["對應速率路線編號"].apply(lambda x: x[1:2] if len(x) > 1 else "")

    # output_df['對應速率路線編號'] = output_df['資料來源'].str[1:2]
    output_df['對應速率路線編號'] = output_df['對應速率路線編號'].astype('int64')
    output_df = output_df.reindex(columns = ['對應速率路線編號','對應速率調查路段編號','對應速率調查方向', '區間', '路段長度', '旅行速率', '服務水準', '時間', '資料來源'])
    output_df['日期'] = output_df['資料來源'].apply(lambda x: os.path.basename(x))
    output_df["日期"] = output_df["日期"].apply(lambda x: x[-8:-4] if len(x) > 8 else "")
    output_df = move_column(output_df, "日期",-2)
    
    return output_df

def speed_organize(speedfolder):
    excelfilelist = findfiles(speedfolder, filetype='.xls') 

    all_combined_df = []
    for excelfile in excelfilelist:
        df = speed_organize_step1(excelfile)
        all_combined_df.append(df)

    all_combined_df = pd.concat(all_combined_df)
    all_combined_df = speed_organize_step2(all_combined_df)

    output_path = os.path.join(speedfolder, '速率資料統整.xlsx')
    investigation_date = os.path.basename(os.path.abspath(os.path.join(speedfolder, '..')))

    all_combined_df.to_excel(output_path, sheet_name=investigation_date, index=False)
    reformat_excel(output_path)

def organize_speedinvestigation_table(step1folder):
    # outputfolder = create_folder(os.path.join('..','01_調查初步分析結果'))
    speedoutputfolder = create_folder(os.path.join(step1folder, '速率'))
    speedoutput_temp_folder = create_folder(os.path.join(speedoutputfolder, 'temp'))

    # speedfolderlist = [r'C:\Users\kjchang\OneDrive - 鼎漢國際工程顧問股份有限公司\台北鼎漢(B_6740)\6798新竹縣關西外環道_運輸需求分析暨補充交通量調查分析案\Technical\交通調查\交通調查公司資料\228連假\速率資料']
    speedfolderlist = find_folder(projectfolder,"速率資料")
    for speedfolder in speedfolderlist:
        speed_organize(speedfolder = speedfolder)

    allspeeddf = read_combined_dataframe(
        findfiles(projectfolder, '速率資料統整.xlsx'))
    allspeeddfoutputfilepath = os.path.join(speedoutput_temp_folder, "01-01_00_調查公司所調查的各路段服務水準.xlsx")
    allspeeddf.to_excel(allspeeddfoutputfilepath, index=False, sheet_name = "各路段服務水準")
    reformat_excel(allspeeddfoutputfilepath)

organize_speedinvestigation_table(step1folder = step1folder)

⚠️ 當發現原本分析的路段，調查公司回傳的資料拆成數個小段，需要將重新計算平均速率  <span style= "color: gray; font-size: 12px;"> (*距離加總 / 時間加總 = 平均速率*) <span> 
       
需要先盤點出對應的路段資料如： * '~Technical/交通調查/速率調查路段.xlsx'*   

In [5]:
def speed_organize_data(speedfolder = r'C:\Users\kjchang\OneDrive - 鼎漢國際工程顧問股份有限公司\台北鼎漢(B_6740)\6798新竹縣關西外環道_運輸需求分析暨補充交通量調查分析案\Technical\交通調查\02_交通調查公司資料\228連假\速率資料'):
    excelfilelist = findfiles(speedfolder, filetype='.xls')
    # excelfile = excelfilelist[0]
    allfile = []
    for excelfile in excelfilelist:
        # filename = get_filename(excelfile)
        filename = get_filename_withoutprojectname(excelfile, step=0)
        sheetnamelist  = get_excel_sheet_names(excelfile)
        sheetnamelist = [sheet for sheet in sheetnamelist if '資料分析' in sheet]
        for sheetname in sheetnamelist:
            # sheetname = sheetnamelist[0]
            columns = [
                "路段編號", "路口起點", "路口迄點", "路線長度", "旅行速率", "行駛速率", 
                "旅行時間", "行駛時間", "延滯時間",  
                "路段延滯_阻塞", "路段延滯_公車停靠", "路段延滯_計程車停靠", "路段延滯_路邊停靠", "路段延滯_行人穿越", "路段延滯_其他", 
                "路口延滯_紅燈",  "路口延滯_左轉同向", "路口延滯_左轉對向", "路口延滯_右轉", "路口延滯_橫越車輛", "路口延滯_行人", "路口延滯_其他"
            ]


            df = pd.read_excel(excelfile , sheet_name=sheetname, skiprows=2)
            df_0 = df.iloc[:20,0:len(columns)] # 20 是因為這次每次只有一條旅行速率調查路線最多拆20個路段，所以請大家隨檔案格式調整
            df_0.columns = columns
            df_0 = df_0[(df_0['路口迄點']!=0) & (df_0['路口迄點']!='合計')]
            df_0['方向'] = 0
            df_0['時段'] = sheetname
            df_0['Source'] = filename

            df_1 = df.iloc[24:44,0:len(columns)] # 20 是因為這次每次只有一條旅行速率調查路線最多拆20個路段，所以請大家隨檔案格式調整
            df_1.columns = columns
            df_1 = df_1[(df_1['路口迄點']!=0) & (df_1['路口迄點']!='合計')]
            df_1['方向'] = 1
            df_1['時段'] = sheetname
            df_1['Source'] = filename

            allfile.append(df_0)
            allfile.append(df_1)

    speed_df = pd.concat(allfile)
    speed_df['時段'] = speed_df['時段'].str.replace('資料分析(','')
    speed_df['時段'] = speed_df['時段'].str.replace(')', '')

    speed_df["對應速率路線編號"] = speed_df["Source"].apply(lambda x: os.path.basename(x))
    speed_df["對應速率路線編號"] = speed_df["對應速率路線編號"].apply(lambda x: x[1:2] if len(x) > 1 else "")

    # speed_df['對應速率路線編號'] = speed_df['Source'].str[1:2]
    speed_df["調查日期"] = speed_df["Source"].apply(lambda x: os.path.basename(x))
    speed_df['調查日期'] = speed_df['調查日期'].str[-8:-4]
    speed_df = move_column(speed_df,'對應速率路線編號',0)
    speed_df = move_column(speed_df,'時段',2)
    speed_df = move_column(speed_df,'調查日期',0)
    speed_df = speed_df.rename(columns = {'路段編號':'對應速率調查路段編號'})
    speed_df[['對應速率路線編號', '對應速率調查路段編號']] = speed_df[['對應速率路線編號', '對應速率調查路段編號']].astype('int64')
    output_path = os.path.join(speedfolder, '速率詳細資料.xlsx')
    speed_df.to_excel(output_path, index= False)
    reformat_excel(output_path)
    return speed_df

def speed_data_reorganize(speed_df, newpairexcel):
    # get_excel_sheet_names(excelpath)
    df = pd.read_excel(newpairexcel, sheet_name='速率調查路段')
    df = df[['對應速率路線編號', '對應速率調查路段編號', '分析路段編號']]
    # df.columns = ['對應速率路線編號', '對應速率調查路段編號', '分析路段編號']
    speed_data = pd.merge(speed_df, df, on = ['對應速率路線編號', '對應速率調查路段編號'], how='outer')
    speed_data = move_column(speed_data, '分析路段編號', 2)
    speed_data = move_column(speed_data, '方向', 3)
    # df = pd.read_excel(excelpath, sheet_name='速率調查路段')
    # df = get_seperatedcolumns_df(excelpath=excelpath, sheetname = '速率調查路段')
    output = speed_data.groupby(['調查日期','時段', '對應速率路線編號', '分析路段編號', '方向', ]).agg({'路線長度':'sum',
                                                                                '行駛時間':'sum', 
                                                                                '延滯時間':'sum',
                                                                                '旅行時間':'sum',
                                                                                '路線長度':'sum'}).reset_index()

    output['旅行速率'] = (output['路線長度'] / output['旅行時間'].astype(float).replace(0, np.nan)) * 3.6
    output['旅行速限'] = 50

    output = get_VL1(output, Vcolumn='旅行速率', VLimitcolumn='旅行速限')
    output = get_VL2(output, Vcolumn='旅行速率', VLimitcolumn='旅行速限')
    # output = get_VL2(output, Vcolumn='旅行速率', VLimitcolumn='旅行速限')
    return output

def re_calculate_speed():
    # 先設定輸出資料夾
    speedoutputfolder = create_folder(os.path.join(step1folder, '速率'))
    speedoutput_temp_folder = create_folder(os.path.join(speedoutputfolder, 'temp'))

    # 交通公司提供的速率資料
    speedfolderlist = find_folder(projectfolder,"速率資料")

    for speedfolder in speedfolderlist:
        # 調整好對應的路段配對資料
        newpairexcel = os.path.join(planfolder, "速率調查路段.xlsx")

        # Step1: 依照"集賢調查公司" 格式，讀取所有speedfolder內的資料
        speed_df = speed_organize_data(speedfolder=speedfolder) 

        survey_date = os.path.basename(os.path.abspath(os.path.join(speedfolder,'..'))) # 取得調查時間 (因檔案命名 調查時間 -> 速率調查/路口轉向量調查/路段交通量調查 -> 細部資料)
        speed_df.to_excel(os.path.join(speedoutput_temp_folder,f'01-01_01_原始資料彙整_{survey_date}.xlsx'), index = False)
        reformat_excel(os.path.join(speedoutput_temp_folder,f'01-01_01_原始資料彙整_{survey_date}.xlsx'))

        # Step2: 因為速率調查路段和定義路段不同，所以要進行長度、時間加總，重新計算服務水準
        output = speed_data_reorganize(speed_df=speed_df, 
                                    newpairexcel=newpairexcel)
        outputfilename = f'01-01_02_速率調查資料_{survey_date}.xlsx'
        output.to_excel(os.path.join(speedoutput_temp_folder, outputfilename),
                        index = False)
        reformat_excel(os.path.join(speedoutput_temp_folder, outputfilename))

    # step3 : 找到所有含有  "02_速率調查資料" 的資料進行整併
    speeddatafilelist = filter_basename(findfiles(speedoutput_temp_folder, 'xlsx'), ['01-01_02_速率調查資料'])
    df_speed_combined = read_combined_dataframe(speeddatafilelist)
    df_speed_combined_path = os.path.join(speedoutputfolder,  '01-01_速率調查資料彙整.xlsx')
    df_speed_combined.to_excel(df_speed_combined_path, index=False)
    reformat_excel(df_speed_combined_path)

re_calculate_speed()

## 01-02 交通量調查

### 路段交通量調查

In [6]:
def combined_section_data(investigatefolder, step1folder, PCE):

    roadsectionfolderlist = find_folder(folderpath= investigatefolder, find_by='路段')
    volumefolder = create_folder(os.path.join(step1folder, '交通量'))
    roadsectionoutputfolder = create_folder(os.path.join(volumefolder, '路段'))

    for roadsectionfolder in roadsectionfolderlist:
        roadsectionfilelist = findfiles(roadsectionfolder, '.xlsx')

        # Step1: 讀取各個方向的路段交通量並合併
        roadsectiondata = []
        for roadsectionfile in roadsectionfilelist:

            rename_columns = ['開始時間', '結束時間', '大貨車_A', '大客車_A', '小型車_A', '機車_A', '大貨車_B', '大客車_B', '小型車_B', '機車_B'] # 需視運具修正
            filename = get_filename(roadsectionfile)
            source = get_filename_withoutprojectname(roadsectionfile, step=0)
            datadate = filename[-5:-1]

            df = pd.read_excel(roadsectionfile, sheet_name='調查')
            first_direction = df.iloc[0,2][:2]
            second_direction = df.iloc[0, 6][:2]

            df1 = pd.read_excel(roadsectionfile, sheet_name='調查', skiprows=3)
            df1 = df1.iloc[:, :10]
            df1.columns = rename_columns
            df1 = df1[['開始時間', '結束時間', '大貨車_A', '大客車_A', '小型車_A', '機車_A']]
            df1.columns = df1.columns.str.replace('_A', '', regex=True)
            df1['方向'] = first_direction
            df1 = move_column(df1, '方向',0)

            df2 = pd.read_excel(roadsectionfile, sheet_name='調查', skiprows=3)
            df2 = df2.iloc[:, :10]
            df2.columns = rename_columns
            df2 = df2[['開始時間', '結束時間', '大貨車_B', '大客車_B', '小型車_B', '機車_B']]
            df2.columns = df2.columns.str.replace('_B', '', regex=True)
            df2['方向'] = second_direction
            df2 = move_column(df2, '方向',0)

            df = pd.concat([df1, df2])
            df['路段'] = filename
            df['Source'] = source
            df['調查日期'] = datadate
            df['調查日期'] = df['調查日期'].astype(str)
            df = move_column(df=df, column_name='路段', insert_index=0)
            df = move_column(df, '調查日期', 1)

            roadsectiondata.append(df)

        # Step2 : 整併後計算PCU、Volume 並輸出
        roadsection_df = pd.concat(roadsectiondata, ignore_index=True)
        roadsection_df['調查路段編號'] = roadsection_df['路段'].str.extract(r'^(\d+)') # 提取最前面的數字（每個公司命名規則需要進行修改）
        roadsection_df = move_column(roadsection_df, '調查路段編號', 0)
        roadsection_df['路段'] = roadsection_df['路段'].str.extract(r'^\d+(.*?)(?:_\(\w+\)\(\d+\))?$')# 提取數字和＿之間的文字（每個公司命名規則需要進行修改）
        roadsection_df['Volume'] = roadsection_df['大貨車']+ roadsection_df['大客車']  + roadsection_df['小型車'] + roadsection_df['機車'] 
        roadsection_df['PCU'] = roadsection_df['大貨車'] * PCE['大貨車'] + roadsection_df['大客車'] * PCE['大客車'] + roadsection_df['小型車'] * PCE['小型車'] + roadsection_df['機車'] * PCE['機車']
        roadsection_df = move_column(roadsection_df, 'Source', roadsection_df.shape[1] - 1)
        roadsectionoutput_temp_folder = create_folder(os.path.join(roadsectionoutputfolder, 'temp'))

        for i in list(roadsection_df['調查日期'].unique()):
            outputdf = roadsection_df[roadsection_df['調查日期'] == i]

            outputdf2 = outputdf.copy()
            outputdf2['小時'] = outputdf2['開始時間'].str[:2].astype('int64')
            outputdf2 = outputdf2.groupby(['調查路段編號', '調查日期', '方向','小時']).agg({'大貨車':'sum', '大客車':'sum', '小型車':'sum', '機車':'sum', 'Volume':'sum','PCU':'sum'}).reset_index()

            roadsectionoutputpath = os.path.join(roadsectionoutput_temp_folder, f'路段交通量資料_{i}.xlsx')
            # outputdf.to_excel(roadsectionoutputpath, index = False, sheet_name=i)
            # outputdf2.to_excel(roadsectionoutputpath, index = False, sheet_name=f'{i}_分時資料')

            with pd.ExcelWriter(roadsectionoutputpath, engine='openpyxl') as writer:
                outputdf.to_excel(writer, index=False, sheet_name='每15分鐘資料')
                outputdf2.to_excel(writer, index=False, sheet_name=f'分時資料')
                pd.DataFrame(PCE, index=['PCE']).to_excel(writer, index=False, sheet_name=f'PCE')  

            reformat_excel(roadsectionoutputpath)


        # Step3: 彙整所有資料
        combinedfile = []
        for i in findfiles(roadsectionoutput_temp_folder, '.xlsx'):
            df = pd.read_excel(i, sheet_name='分時資料')
            combinedfile.append(df)
        combinedfile = pd.concat(combinedfile, ignore_index=True)
        combinedpath = os.path.join(roadsectionoutputfolder, '01-02_路段彙整.xlsx')


        # Step4: 需要區分出尖峰時段資料
        group_by = ['調查路段編號', '調查日期', '方向']
        sum_by = 'PCU'
        hourcolumn = '小時'
        combinedfile_am, combinedfile_pm = get_peak_AMPM(df = combinedfile, group_by = group_by, sum_by = sum_by, hourcolumn = hourcolumn)
        combinedfile_am['時段'] = '晨峰'
        combinedfile_pm['時段'] = '昏峰'
        combinedfile_am = move_column(combinedfile_am, '時段', 2)
        combinedfile_pm = move_column(combinedfile_pm, '時段', 2)
        combinedfile_am = combinedfile_am.rename(columns = {'晨峰小時PCU':'PCU'})
        combinedfile_pm = combinedfile_pm.rename(columns = {'昏峰小時PCU':'PCU'})
        combinedfile_peak_df = pd.concat([combinedfile_am, combinedfile_pm], ignore_index=True)

        with pd.ExcelWriter(combinedpath, engine='openpyxl') as writer:
            combinedfile.to_excel(writer, index=False, sheet_name='分時資料')
            combinedfile_peak_df.to_excel(writer, index=False, sheet_name='尖峰小時資料')
            get_peak_percent(df= combinedfile, group_by=group_by, sum_by=sum_by, hourcolumn=hourcolumn).to_excel(writer, index=False, sheet_name='尖峰率資料')
            pd.DataFrame(PCE, index=['PCE']).to_excel(writer, index=False, sheet_name=f'PCE')   # 提供核對用的PCE
        reformat_excel(combinedpath)

combined_section_data(investigatefolder=investigatefolder, step1folder=step1folder, PCE = PCE)

### 路口轉向量調查

#### FROM

In [7]:
def combined_fromdirection_vehicles(df):
    target_cols = df.filter(regex='大車|小車|機車兩段|機車').columns.tolist()

    df[target_cols] = df[target_cols].replace('-', 0)
    df[target_cols] = df[target_cols].astype(int)

    # 先建立分類關鍵字
    vehicle_groups = {
        '大車': '大車總數',
        '小車': '小車總數',
        '機車兩段': '機車兩段總數',
        '機車': '機車總數'
    }

    # 逐類型篩選欄位並加總
    for vehicle, new_col in vehicle_groups.items():
        # 注意順序！要先排除「機車兩段」被「機車」吃掉的問題
        if vehicle == '機車':
            # 過濾「機車」但排除「機車兩段」
            target_cols = [col for col in df.columns if '機車' in col and '機車兩段' not in col]
        else:
            target_cols = [col for col in df.columns if vehicle in col]
        
        # 加總對應欄位
        df[new_col] = df[target_cols].sum(axis=1)


    df = move_column(df, '大車總數', 5)
    df = move_column(df, '小車總數', 6)
    df = move_column(df, '機車總數', 7)
    df = move_column(df, '機車兩段總數', 8)


    return df 

def combined_todirection_vehicles(intersectiondata_df):
    df = intersectiondata_df.copy()

    # 方向列表 & 車種列表
    Directionlist = ['A', 'B', 'C', 'D']
    Vehiclelist = ['大車', '小車', '機車兩段', '機車']


    target_cols = df.filter(regex='大車|小車|機車兩段|機車').columns.tolist()

    df[target_cols] = df[target_cols].replace('-', 0)
    df[target_cols] = df[target_cols].astype(int)

    # 動態生成所有需要轉換的欄位名稱
    all_columns = [f"{v}{d}" for v in Vehiclelist for d in Directionlist]

    # 使用 melt 將各個車種展開
    df_melted = df.melt(
        id_vars=['路口調查點位編號', '調查日期', '起始時間', '結束時間'],
        value_vars=all_columns,
        var_name='車種方向',
        value_name='數量'
    )

    # 提取 "抵達方向" (A, B, C, D)
    df_melted['抵達方向'] = df_melted['車種方向'].str[-1]

    # 提取 "車種"
    df_melted['車種'] = df_melted['車種方向'].str[:-1]

    # 重新整理數據，使不同車種成為獨立的欄位
    df_melted = df_melted.pivot_table(
        index=['路口調查點位編號', '調查日期', '起始時間', '結束時間', '抵達方向'],
        columns='車種',
        values='數量',
        fill_value=0
    ).reset_index()

    # 取消多層索引並確保欄位名稱正確
    df_melted.columns.name = None
    return df_melted

def get_intersectionfiledata(intersectionfilelist):
    # step1讀取資料
    intersectiondata = [] 
    for intersectionfile in  intersectionfilelist:
        source = get_filename_withoutprojectname(intersectionfile)
        filename = get_filename(intersectionfile)
        datadate = filename[-5:-1]
        intersectionnode = int(filename[:1])

        df = pd.read_excel(intersectionfile, sheet_name='轉向量')
        df['路口調查點位編號'] = intersectionnode
        df['調查日期'] = datadate
        df['Source'] = source
        df = move_column(df, '調查日期',0 )
        df = move_column(df , '路口調查點位編號', 0)
        intersectiondata.append(df)
    
    return pd.concat(intersectiondata)

In [8]:
def organize_intersection_from_data(investigatefolder, step1folder, PCE):

    volumefolder = create_folder(os.path.join(step1folder, '交通量'))
    intersectionoutputfolder = create_folder(os.path.join(volumefolder, '路口'))

    intersectionfolderlist = find_folder(investigatefolder, '路口')


    # step1讀取資料
    intersectionfilelist = [] # 把所有調查資料中，不同時段的「路口」調查資料中的資料都找出來
    for intersectionfolder in intersectionfolderlist:
        intersectionfilelist.extend(findfiles(intersectionfolder, '.xlsx'))

    intersectiondata_df = get_intersectionfiledata(intersectionfilelist)


    # # step2 統整源自各個方向的交通量
    intersectiondata_df = combined_fromdirection_vehicles(intersectiondata_df)
    intersectiondata_df['Volume'] = intersectiondata_df['大車總數']+ intersectiondata_df['小車總數']  + intersectiondata_df['機車總數'] + intersectiondata_df['機車兩段總數'] 
    intersectiondata_df['PCU'] = intersectiondata_df['大車總數'] * PCE['大型車'] + intersectiondata_df['小車總數'] * PCE['小型車'] + intersectiondata_df['機車總數'] * PCE['機車'] + intersectiondata_df['機車兩段總數'] * PCE['機車']

    intersectionoutput_temp_folder = create_folder(os.path.join(intersectionoutputfolder,'temp'))

    combinedfile = []
    for i in list(intersectiondata_df['調查日期'].unique()):
        outputdf = intersectiondata_df[intersectiondata_df['調查日期'] == i]
        intersectionoutputpath = os.path.join(intersectionoutput_temp_folder, f'01-02_FROM路口交通量資料_{i}.xlsx')

        outputdf2 = outputdf.copy()
        outputdf2['起始時間'] = outputdf2['起始時間'].astype(str)
        outputdf2['小時'] = outputdf2['起始時間'].str[:2].astype('int64')
        outputdf2 = outputdf2.groupby(['路口調查點位編號', '調查日期','臨近路段','小時']).agg({'大車總數':'sum', '小車總數':'sum', '機車總數':'sum', '機車兩段總數':'sum', "Volume":'sum', 'PCU':'sum'}).reset_index()

        combinedfile.append(outputdf2)
        with pd.ExcelWriter(intersectionoutputpath, engine='openpyxl') as writer:
            outputdf.to_excel(writer, index=False, sheet_name='每15分鐘資料')  # 第一張工作表
            outputdf2.to_excel(writer, index=False, sheet_name=f'分時資料')  # 第二張工作表

    # # step3 合併每日資料
    # combinedfile = []
    # for i in filter_basename(findfiles(intersectionoutput_temp_folder, '.xlsx'), ['FROM']):
    #     df = pd.read_excel(i, sheet_name='分時資料')
    #     combinedfile.append(df)
    combinedfile = pd.concat(combinedfile, ignore_index=True)
    combinedpath = os.path.join(intersectionoutputfolder, '01-02_FROM路口彙整.xlsx')

    # Step4: 需要區分出尖峰時段資料
    group_by = ['路口調查點位編號', '調查日期', '臨近路段']
    sum_by = 'PCU'
    hourcolumn = '小時'
    combinedfile_am, combinedfile_pm = get_peak_AMPM(df = combinedfile, group_by = group_by, sum_by = sum_by, hourcolumn = hourcolumn)
    combinedfile_am['時段'] = '晨峰'
    combinedfile_pm['時段'] = '昏峰'
    combinedfile_am = move_column(combinedfile_am, '時段', 2)
    combinedfile_pm = move_column(combinedfile_pm, '時段', 2)
    combinedfile_am = combinedfile_am.rename(columns = {'晨峰小時PCU':'PCU'})
    combinedfile_pm = combinedfile_pm.rename(columns = {'昏峰小時PCU':'PCU'})
    combinedfile_peak_df = pd.concat([combinedfile_am, combinedfile_pm], ignore_index=True)

    with pd.ExcelWriter(combinedpath, engine='openpyxl') as writer:
        combinedfile.to_excel(writer, index=False, sheet_name='分時資料')
        combinedfile_peak_df.to_excel(writer, index=False, sheet_name='尖峰小時資料')
        get_peak_percent(df= combinedfile, group_by=group_by, sum_by=sum_by, hourcolumn=hourcolumn).to_excel(writer, index=False, sheet_name='尖峰率資料')
        pd.DataFrame(PCE, index=['PCE']).to_excel(writer, index=False, sheet_name=f'PCE')   # 提供核對用的PCE
    reformat_excel(combinedpath)

organize_intersection_from_data(investigatefolder = investigatefolder, step1folder = step1folder, PCE = PCE)

/var/folders/1p/d63vtkgd60d4sbpl791031gw0000gn/T/ipykernel_32184/237934095.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[target_cols] = df[target_cols].replace('-', 0)


#### TO

In [9]:
def organize_intersection_to_data(investigatefolder, step1folder, PCE):

    volumefolder = create_folder(os.path.join(step1folder, '交通量'))
    intersectionoutputfolder = create_folder(os.path.join(volumefolder, '路口'))

    intersectionfolderlist = find_folder(investigatefolder, '路口')


    # step1讀取資料
    intersectionfilelist = [] # 把所有調查資料中，不同時段的「路口」調查資料中的資料都找出來
    for intersectionfolder in intersectionfolderlist:
        intersectionfilelist.extend(findfiles(intersectionfolder, '.xlsx'))

    intersectiondata_df = get_intersectionfiledata(intersectionfilelist)


    # # step2 統整源自各個方向的交通量
    intersectiondata_df_to = combined_todirection_vehicles(intersectiondata_df)
    intersectiondata_df_to = intersectiondata_df_to.rename(columns = {'大車':'大車總數', '小車':'小車總數', '機車兩段':'機車兩段總數', '機車':'機車總數'})
    intersectiondata_df_to['Volume'] = intersectiondata_df_to['大車總數']  + intersectiondata_df_to['小車總數']  + intersectiondata_df_to['機車總數'] + intersectiondata_df_to['機車兩段總數'] 
    intersectiondata_df_to['PCU'] = intersectiondata_df_to['大車總數'] * PCE['大型車'] + intersectiondata_df_to['小車總數'] * PCE['小型車'] + intersectiondata_df_to['機車總數'] * PCE['機車'] + intersectiondata_df_to['機車兩段總數'] * PCE['機車']

    intersectionoutput_temp_folder = create_folder(os.path.join(intersectionoutputfolder,'temp'))

    combinedfile = []

    for i in list(intersectiondata_df_to['調查日期'].unique()):
        outputdf = intersectiondata_df_to[intersectiondata_df_to['調查日期'] == i]
        intersectionoutputpath = os.path.join(intersectionoutput_temp_folder, f'01-02_TO路口交通量資料_{i}.xlsx')

        outputdf2 = outputdf.copy()
        outputdf2['起始時間'] = outputdf2['起始時間'].astype(str)
        outputdf2['小時'] = outputdf2['起始時間'].str[:2].astype('int64')
        outputdf2 = outputdf2.groupby(['路口調查點位編號', '調查日期','抵達方向','小時']).agg({'大車總數':'sum', '小車總數':'sum', '機車總數':'sum', '機車兩段總數':'sum', "Volume":'sum', 'PCU':'sum'}).reset_index()
        combinedfile.append(outputdf2)

        with pd.ExcelWriter(intersectionoutputpath, engine='openpyxl') as writer:
            outputdf.to_excel(writer, index=False, sheet_name='每15分鐘資料')  # 第一張工作表
            outputdf2.to_excel(writer, index=False, sheet_name=f'分時資料')  # 第二張工作表



    # step3 合併每日資料
    combinedfile = pd.concat(combinedfile, ignore_index=True)
    combinedpath = os.path.join(intersectionoutputfolder, '01-02_TO路口彙整.xlsx')

    # Step4: 需要區分出尖峰時段資料
    group_by = ['路口調查點位編號', '調查日期', '抵達方向']
    sum_by = 'PCU'
    hourcolumn = '小時'
    combinedfile_am, combinedfile_pm = get_peak_AMPM(df = combinedfile, group_by = group_by, sum_by = sum_by, hourcolumn = hourcolumn)
    combinedfile_am['時段'] = '晨峰'
    combinedfile_pm['時段'] = '昏峰'
    combinedfile_am = move_column(combinedfile_am, '時段', 2)
    combinedfile_pm = move_column(combinedfile_pm, '時段', 2)
    combinedfile_am = combinedfile_am.rename(columns = {'晨峰小時PCU':'PCU'})
    combinedfile_pm = combinedfile_pm.rename(columns = {'昏峰小時PCU':'PCU'})
    combinedfile_peak_df = pd.concat([combinedfile_am, combinedfile_pm], ignore_index=True)

    with pd.ExcelWriter(combinedpath, engine='openpyxl') as writer:
        combinedfile.to_excel(writer, index=False, sheet_name='分時資料')
        combinedfile_peak_df.to_excel(writer, index=False, sheet_name='尖峰小時資料')
        get_peak_percent(df= combinedfile, group_by=group_by, sum_by=sum_by, hourcolumn=hourcolumn).to_excel(writer, index=False, sheet_name='尖峰率資料')
        pd.DataFrame(PCE, index=['PCE']).to_excel(writer, index=False, sheet_name=f'PCE')   # 提供核對用的PCE
    reformat_excel(combinedpath)

organize_intersection_to_data(investigatefolder = investigatefolder, step1folder = step1folder, PCE = PCE)

/var/folders/1p/d63vtkgd60d4sbpl791031gw0000gn/T/ipykernel_32184/237934095.py:46: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[target_cols] = df[target_cols].replace('-', 0)


## 01_03 手動檢核提供資料是否有誤 

checklist : 
1. 確認路口轉向量資料：是否有在不合理的地方有量
2. 確認路段尖峰小時是否有一致性

# 02_整合調查資料為轉向量資料（較多視專案需求處理）

## 02-01 速率資料比對至路段

In [10]:
comparisontable = pd.read_excel(os.path.join(planfolder, "交通調查點位.xlsx"), sheet_name='路段對照表', skiprows=1)
comparisontable = comparisontable.iloc[1:,:]
comparisontable_reindex_columns = ['路段編號', '道路名稱', '起訖', '方向', '對應速率路線編號', '分析路段編號', '對應速率調查方向']
comparisontable = comparisontable.reindex(columns = comparisontable_reindex_columns)
comparisontable = comparisontable[comparisontable['對應速率路線編號'] != 0]

speedfilepath = os.path.join(step1folder, '速率', '01-01_速率調查資料彙整.xlsx')
df_speed = pd.read_excel(speedfilepath)
df_speed = df_speed.rename(columns = {'方向':'對應速率調查方向'})
df_speedout = pd.merge(comparisontable, df_speed, on = ['對應速率路線編號', '分析路段編號', '對應速率調查方向'])

df_speedoutput_path = os.path.join(step2folder, '02-01_調查速率與對應路段.xlsx')
df_speedout.to_excel(df_speedoutput_path,sheet_name='尖峰資料',index = False)
reformat_excel(df_speedoutput_path)

/Users/zhangkaijie/Library/Python/3.9/lib/python/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## 02_02 把路口轉向量資料跟路段交通量皆轉為指定路段交通量資料

In [4]:
def get_volume_from_and_to_file(volume01_folder,sheetname = '分時資料' ):
    from_path = os.path.join(volume01_folder,"路口", "01-02_FROM路口彙整.xlsx")
    df_from = pd.read_excel(from_path, sheet_name=sheetname)
    df_from['流入流出'] = 'FROM'
    df_from = df_from.rename(columns={'臨近路段':'路口對應方向'})

    to_path = os.path.join(volume01_folder,"路口", "01-02_TO路口彙整.xlsx")
    df_to = pd.read_excel(to_path, sheet_name=sheetname)
    df_to['流入流出'] = 'TO'
    df_to = df_to.rename(columns={'抵達方向':'路口對應方向'})

    df = pd.concat([df_to, df_from], ignore_index=True)
    try:
        df = move_column(df, 'InOrOut',2)
    except:
        pass
    return df

In [5]:
# step1 : 讀取對照表以及儲存資料位置
comparisontable = pd.read_excel(os.path.join(planfolder, "交通調查點位.xlsx"), sheet_name='路段對照表', skiprows=1)
comparisontable = comparisontable.iloc[1:,:]

comparisontable_intersection = comparisontable[comparisontable['點位調查方法'].str.contains('路口', na=False)]
comparisontable_section = comparisontable[comparisontable['點位調查方法'].str.contains('路段', na=False)]


# step2 : 讀取路口資料
volume01_folder = os.path.join(os.path.join(step1folder,"交通量"))
df_intersection_volume = get_volume_from_and_to_file(volume01_folder=volume01_folder, sheetname='分時資料')
df_intersection_volume = df_intersection_volume.rename(columns = {'路口調查點位編號':'對應調查點位'})
df_intersection_volume = pd.merge(comparisontable_intersection[['路段編號','道路名稱', 	'起訖', '方向', '對應調查點位','點位調查方法', '流入流出', '路口對應方向']],
                                  df_intersection_volume,
                                  on=['對應調查點位', '流入流出', '路口對應方向'])

# # step3 : 讀取路段資料
df_section_volume = pd.read_excel(os.path.join(volume01_folder, '路段', '01-02_路段彙整.xlsx'))
df_section_volume = df_section_volume.rename(columns = {'調查路段編號':'對應調查點位'})
df_section_volume = pd.merge(comparisontable_section[['路段編號','道路名稱', '起訖', '方向', '對應調查點位','點位調查方法']], 
                             df_section_volume, 
                             on = ['對應調查點位', '方向'])

df_section_volume['大車總數'] = df_section_volume['大貨車'] + df_section_volume['大客車']
df_section_volume = df_section_volume.rename(columns = {'小型車':'小車總數', '機車':'機車總數'}) 

# step4 : 整合
df_volume = pd.concat([df_intersection_volume, df_section_volume])
volumeoutputpath = os.path.join(step2folder, '02-02_整理各路段交通量及PCU.xlsx')

group_by = ['調查日期','路段編號', '道路名稱', '起訖', '方向']
sum_by='PCU'
hourcolumn='小時'

df_volume_am, df_volume_pm = get_peak_AMPM(df = df_volume, group_by = group_by, sum_by = sum_by, hourcolumn = hourcolumn)
df_volume_am['時段'] = '晨峰'
df_volume_pm['時段'] = '昏峰'
df_volume_am = move_column(df_volume_am, '時段', 2)
df_volume_pm = move_column(df_volume_pm, '時段', 2)
df_volume_am = df_volume_am.rename(columns = {'晨峰小時PCU':'PCU'})
df_volume_pm = df_volume_pm.rename(columns = {'昏峰小時PCU':'PCU'})
df_volume_peak_df = pd.concat([df_volume_am, df_volume_pm], ignore_index=True)

with pd.ExcelWriter(volumeoutputpath, engine='openpyxl') as writer:
    df_volume.to_excel(writer, index=False, sheet_name='各路段分時資料')
    df_volume_peak_df.to_excel(writer, index=False, sheet_name='尖峰小時資料')
    get_peak_percent(df = df_volume, group_by = ['路段編號', '道路名稱', '起訖', '方向'], sum_by='PCU', hourcolumn='小時').to_excel(writer, index=False, sheet_name='尖峰率資料')
    pd.DataFrame(PCE, index=['PCE']).to_excel(writer, index=False, sheet_name=f'PCE')   # 提供核對用的PCE

reformat_excel(volumeoutputpath)

C:\Users\kjchang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


## 02_03 速率合併資料


In [ ]:
def combined_volume_speed(step2folder, planfolder):
    step2folder = os.path.abspath(os.path.join(os.getcwd(), '..','02_整併交通量資料'))

    speedpath = os.path.join(step2folder, '02-01_調查速率與對應路段.xlsx')
    volumepath = os.path.join(step2folder, "02-02_整理各路段交通量及PCU.xlsx")

    df_speed = pd.read_excel(speedpath, sheet_name="尖峰資料")
    df_volume = pd.read_excel(volumepath, sheet_name="尖峰小時資料")

    # 需要調整欄位資料
    # 1. 調查日期維持四位文字 -> 後續好對照
    # 2. 因集賢公司的工作頁籤為"平(假)日晨(昏)峰" 因此在速率調查時段的欄位 需要改為"晨/昏峰"
    df_speed["調查日期"] = df_speed["調查日期"].astype(str).str.zfill(4)  # 確保是四位數
    df_volume["調查日期"] = df_volume["調查日期"].astype(str).str.zfill(4)  # 確保是四位數
    df_speed['時段'] = df_speed['時段'].str[-2:]

    df_combined = pd.merge(df_volume, df_speed, on = ['路段編號', '道路名稱', '起訖', '方向', '調查日期', '時段'], how='outer') 
    df_combined_path = os.path.join(step2folder, '02-03_路段對應調查資料彙整.xlsx')
    df_combined.to_excel(df_combined_path, index=False, sheet_name="路段應調查資料彙整")
    reformat_excel(df_combined_path)
    return df_combined

df_combined = combined_volume_speed(step2folder, planfolder )

In [ ]:
pasteinpath = findfiles(planfolder, '填表範例.xlsx')[0] #檔名需要為「填表範例結尾」，可以視專案進行調整，或是直接寫絕對路徑
pasteonfilename = '交通服務水準彙整'
pasteonpath = os.path.join(create_folder(os.path.join(step2folder, '輸出分析檔案')),f'{pasteonfilename}.xlsx')
copyfile(originalpath= pasteinpath, newpath = pasteonpath)

↓目前這個寫法確實可以貼上資料，但是因為我的其他欄位有的是對照其他資料來源的，他資料表格會失效

In [55]:
df_combined[['調查日期', '時段']].drop_duplicates()

,調查日期,時段
0,0228,昏峰
1,0228,晨峰
2,0301,昏峰
3,0301,晨峰
4,0302,昏峰
5,0302,晨峰


In [58]:
unique_dates_periods = df_combined[['調查日期', '時段']].drop_duplicates()

for date, period in unique_dates_periods.itertuples(index=False):
    sheetnametext = f'{date}_{period}'
    print(sheetnametext)
    print(date)
    print(period)
    print("=======")

0228_昏峰
0228
昏峰
0228_晨峰
0228
晨峰
0301_昏峰
0301
昏峰
0301_晨峰
0301
晨峰
0302_昏峰
0302
昏峰
0302_晨峰
0302
晨峰


In [57]:
unique_dates_periods = df_combined[['調查日期', '時段']].drop_duplicates()

for date, period in unique_dates_periods.itertuples(index=False):
    sheetnametext = f'{date}_{period}'
    df_filtered = df_combined[(df_combined['調查日期'] == date) & (df_combined['時段'] == period)]
    print(f"調查日期: {date}, 時段: {period}")
    print(df_filtered)


調查日期: 0228, 時段: 昏峰
     路段編號  道路名稱  時段            起訖  方向  對應調查點位   點位調查方法  流入流出 路口對應方向  調查日期  \
0       1   正義路  昏峰       關西交流道以西  往東       1  路口轉向量調查  FROM      C  0228   
6       2   正義路  昏峰       關西交流道以西  往西       1  路口轉向量調查    TO      C  0228   
12      3   正義路  昏峰       關西交流道匝道  往南       1  路口轉向量調查  FROM      D  0228   
18      4   正義路  昏峰       關西交流道匝道  往北       1  路口轉向量調查    TO      D  0228   
24      5   正義路  昏峰     關西交流道-北平路  往東       1  路口轉向量調查    TO      A  0228   
30      6   正義路  昏峰     關西交流道-北平路  往西       1  路口轉向量調查  FROM      A  0228   
36      7   正義路  昏峰     北平路 - 光明路  往東       5  路口轉向量調查    TO      A  0228   
42      8   正義路  昏峰     北平路 - 光明路  往西       5  路口轉向量調查  FROM      A  0228   
48      9   正義路  昏峰       光明路-明德路  往南       8  路段交通量調查   NaN    NaN  0228   
54     10   正義路  昏峰       光明路-明德路  往北       8  路段交通量調查   NaN    NaN  0228   
60     11   正義路  昏峰     明德路 - 北平路  往南       2  路口轉向量調查    TO      B  0228   
66     12   正義路  昏峰     明德路 - 北平路  往北       2  路口轉向量調查  F

In [53]:
from openpyxl import load_workbook
import pandas as pd

# 確保 openpyxl 可讀取 Excel
with pd.ExcelWriter(pasteonpath, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    df_combined.to_excel(writer, index=False, sheet_name='調查資料總表')
